In [246]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [247]:
train = pd.read_csv("Trainfile.csv")
test = pd.read_csv("Testfile.csv")
train.head()

,Idx,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,45,RL,59.0,7227,Pave,NaN,Reg,HLS,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,105500
1,2,60,RL,NaN,12227,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,272000
2,3,20,RL,65.0,7800,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2009,WD,Normal,130000
3,4,60,RL,88.0,12665,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2007,WD,Normal,281213
4,5,20,RL,105.0,15431,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2009,WD,Normal,555000


In [248]:
len(train.columns)

81

In [249]:
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ID = train['Idx']
test_ID = test['Idx']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Idx", axis = 1, inplace = True)
test.drop("Idx", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

The train data size before dropping Id feature is : (1138, 81) 
The test data size before dropping Id feature is : (322, 81) 

The train data size after dropping Id feature is : (1138, 80) 
The test data size after dropping Id feature is : (322, 80) 


In [250]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (1460, 79)


In [251]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
print(len(missing_data))
missing_data.head(20)

19


,Missing Ratio
PoolQC,99.520548
MiscFeature,96.301370
Alley,93.767123
Fence,80.753425
FireplaceQu,47.260274
LotFrontage,17.739726
GarageYrBlt,5.547945
GarageType,5.547945
GarageFinish,5.547945
GarageQual,5.547945


In [252]:
all_data.drop(["MiscFeature", "PoolQC", "Alley", "Fence", "FireplaceQu"], axis = 1, inplace = True) 

In [253]:
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [254]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

In [255]:
all_data["GarageYrBlt"] = all_data["GarageYrBlt"].fillna(0)

In [256]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [257]:
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [258]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [259]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

,Missing Ratio


In [260]:
cat_data = all_data.select_dtypes(include=['object']).copy()

In [261]:
from sklearn.preprocessing import LabelEncoder
for c in cat_data:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

Shape all_data: (1460, 74)


In [262]:
print(all_data['Heating'])

0       1
1       1
2       1
3       1
4       1
       ..
1455    1
1456    1
1457    1
1458    1
1459    1
Name: Heating, Length: 1460, dtype: int64


In [263]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [264]:
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,45,3,59.0,7227,1,3,1,0,0,1,...,0,0,0,0,0,0,6,2008,8,4
1,60,3,80.0,12227,1,0,3,0,0,0,...,282,0,0,0,0,0,7,2008,8,4
2,20,3,65.0,7800,1,3,3,0,4,0,...,0,0,0,0,0,0,6,2009,8,4
3,60,3,88.0,12665,1,0,3,0,4,0,...,39,0,0,0,0,0,2,2007,8,4
4,20,3,105.0,15431,1,3,3,0,4,0,...,72,0,0,170,0,0,4,2009,8,4


In [265]:
from sklearn import preprocessing

In [266]:
standardized_X = preprocessing.scale(train)

In [267]:
standardized_X

array([[-0.29690647, -0.05687835, -0.49621426, ...,  0.11529532,
         0.31313362,  0.20016196],
       [ 0.05603179, -0.05687835,  0.41360071, ...,  0.11529532,
         0.31313362,  0.20016196],
       [-0.8851369 , -0.05687835, -0.23626712, ...,  0.86935319,
         0.31313362,  0.20016196],
       ...,
       [-0.8851369 , -0.05687835, -0.01964451, ..., -1.39282043,
         0.31313362,  0.20016196],
       [-0.8851369 , -0.05687835, -0.01964451, ...,  0.86935319,
         0.31313362,  0.20016196],
       [-0.8851369 , -0.05687835,  0.63022332, ...,  0.11529532,
         0.31313362,  0.20016196]])

In [268]:
train_n = pd.read_csv("Trainfile.csv")
Y_pred = np.array(train_n['SalePrice'])

In [269]:
x_t=np.transpose(standardized_X)
max=np.amax(np.log(Y_pred))
y_t=np.log(Y_pred)/max
x_t.shape
print(max)

13.521139497361697


In [270]:
num_examples=1138
num_dimentions=74

In [271]:
def predict(w, b, x):
    """ A method that runs our linear model
    w : np.ndarray (1, num_dimensions)
        A weight matrix which will be fit to the data
    b : float (1, 1)
        The bias unit for our linear model
    x : np.ndarray (num_dimensions, num_examples)
        The m input examples for which we would like to predict a y value
    """
    return w @ x + b

In [272]:
def cost(y, y_pred):
    """
    y : np.ndarray (num_examples, 1)
        The ground truth results
    y_pred : np.ndarray (num_examples, 1)
    """
    return ((y_pred - y) ** 2).mean()

In [273]:
def initialize_parameters(n):
    """
    n : int
        The dimensions of the data
    """
    return np.random.randn(1, n)*0.001 , 0.0

In [274]:
def gradient(x, y, y_pred):
    """
    x : np.ndarray (num_dimensions, num_examples)
        The m input examples for which we would like to predict a y value
    y : np.ndarray (num_examples, 1)
        The ground truth results
    y_pred : np.ndarray (num_examples, 1)
        Our model's predictions
    """
    dw = ((y_pred - y) * x).mean()
    db = (y_pred - y).mean()
    return dw, db

In [275]:
def gradient_descent(x, y, w, b, epochs=1000, learning_rate=0.0025):
    """
    x : np.ndarray (num_dimensions, num_examples)
        The m input examples for which we would like to predict a y value
    y : np.ndarray (num_examples, 1)
        The ground truth results
    w : np.ndarray (1, num_dimensions)
        A weight matrix which will be fit to the data
    b : float
        The bias unit for our linear model
    epochs : int
        The number of times our model iterates over the entire training set
    learning_rate : float
        The rate at which our model tends to adjust it's parameters
    """
    for i in range(epochs):
        # predict the value of y with our current model
        y_pred = predict(w, b, x)
        
        # compute the gradient with respect to w and b
        dw, db = gradient(x, y, y_pred)
        
        # adjust w and b as to minimize the cost function
        w -= learning_rate * dw
        b -= learning_rate * db
        
    return w, b

In [276]:
def model(x, y, epochs=50000, learning_rate=0.0025):

    # initialize w and b
    w, b = initialize_parameters(num_dimentions)
        
    # run gradient descent in order to find optimal values for w and b
    w, b = gradient_descent(x, y, w, b, epochs=epochs, learning_rate=learning_rate)
        
    # calculate the final cost function
    c = cost(y, predict(w, b, x))
    
    # print out the model details
    print('w:', w)
    print('b:', b)
    print('cost:', c)
    
    return w, b

In [277]:
w, b = model(x_t, y_t, epochs=100000, learning_rate=0.0001)

w: [[ 2.61297936e-03  8.82501067e-04  9.00601562e-04  2.12476845e-03
   1.15509568e-03  3.21551276e-03  1.98543932e-03  7.29666475e-04
   1.77977993e-03  2.10886694e-03  1.38280911e-03  3.11714862e-03
   1.90855476e-03  2.42066569e-04  2.51360998e-03  1.38034112e-03
   8.53472497e-04  2.95857286e-03  1.20838693e-03  1.76679213e-03
   1.86163970e-03  1.17503944e-03  2.05553935e-03  1.85277941e-03
   1.66355265e-03  1.90436026e-03  2.25184478e-03  2.34238988e-03
   1.92563619e-03  2.70912728e-03  1.77718118e-03  1.44210734e-03
   8.72809202e-04  2.17986945e-03  2.46022029e-03 -4.20829977e-04
   1.77199552e-03  2.62302980e-03  5.79527159e-04  4.52718624e-04
   1.01818393e-03  1.85497220e-03  2.80346896e-04  1.15396638e-03
   2.35197068e-03  1.57469177e-03  1.43906077e-03  2.85845752e-03
   3.41477167e-03  1.42655691e-03  1.45578025e-03  3.31217575e-03
   2.30001141e-03  1.86778548e-03  2.47609087e-03  2.14576843e-04
   7.67062704e-04  2.06100872e-03  1.13727115e-03  1.41760936e-03
   5.83

In [278]:
y_predict = w @ x_t + b
y_predict= np.exp(y_predict*13.521139497)

In [279]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print(MAPE(Y_pred, y_predict))

21.805515065457747
